In [ ]:
import pydap.client
import requests
import numpy
import configparser

# Part 1: Parsing DMRs
the client has the following functions to open downloaded files
- ```open_file()```
- ```open_dods_file()```
- ```open_dmr_file()```
- ```open_dap_file()```

In [ ]:
fname = 'data/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.dmr'
dataset = pydap.client.open_dmr_file(fname)
dataset['sea_ice_fraction']

# Part 2: Loading DAP files

## Example 1

In [ ]:
fname = 'data/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_subset.dap'
dataset = pydap.client.open_dap_file(fname)
dataset

In [ ]:
dataset['sea_ice_fraction']

In [ ]:
dataset['sea_ice_fraction'].attributes

## Example 2

In [ ]:
fname = 'data/coads_climatology.nc_full.dap'
dataset = pydap.client.open_dap_file(fname)
dataset

In [ ]:
dataset['SST'].array[1, 1:10, 1:10]

In [ ]:
fname = 'data/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf.dap'
pydap.client.open_dap_file(fname)

# Part 3: Accessing OPeNDAP server

## Example 1: Previously inaccesible through pydap
as posted by Alberto Torres in the GH issues https://github.com/pydap/pydap/issues/226#issuecomment-1198288875

The example demonstrates the ability to access

- Data requiring an earthdata loging
- Data that is served up exclusively through DAP4
- Data containing 8, 16, 32 bit integers and 32 bit floats

### Creating a session 
Note:
The username and password are loaded from the file `user.config`. Edit this file accordingly or enter your credentials here directly

In [ ]:
config = configparser.ConfigParser()
config.read('user.config')
username = config['user']['user']
password = config['user']['pwd']

In [ ]:
class SessionEarthData(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
                    redirect_parsed.hostname != self.AUTH_HOST and \
                    original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']
        return

session = SessionEarthData(username=username, password=password)

### Opening the dataset URL to build a dataset
- ```pydap.client.open_url()``` downloads DMR, pareses it, and builds a dataset from it. I.e. interpreting :
    - data types including endianess
    - shapes
    - hierarchy (groups)
    - relations (maps) of variables and dimensions
    - variable attributes
- No data is downloaded; rather, 'DummyData' of the according type and shpae is inserted into the dataset
- Using the dap 4 protocol is specified using either
    - Using the schema 'dap4://' in the url (canonical)
    - Specifing the 'schema' kwarg in the function call: ```pydap.client.open_url(url, protocol='dap4', **kwargs)```.

In [ ]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'opendap.earthdata.nasa.gov'
path = '/collections/C1996881146-POCLOUD/granules/'
dataset = '20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'
dap4_url

#### Accessing with DAP2 
We first demnonstrate that accessing the dataset via dap2 fails:

In [ ]:
# pydap.client.open_url(dap2_url, session=session)

#### Accessing via DAP4

In [ ]:
pydap_ds = pydap.client.open_url(dap4_url, session=session)
#pydap_ds = pydap.client.open_url(dap2_url, session=session, protocol='dap4')
pydap_ds

#### Inspecting the dataset

In [ ]:
pydap_ds._dict

### Accessing the data
We can access the variables attributes. Note that the checksum is part of the data response, not the DMR. 

In [ ]:
pydap_ds['sea_ice_fraction'].attributes

#### Lazy subsetting
- pydap uses numpy-stype fancy indexing; i.e ```[start:stop:step]```
- opendap uses fancy indexing of following style ```[start:step:stop]```
- pydap translates between those styles

In [ ]:
variable = pydap_ds['sea_ice_fraction'][0, 1700:1799:10, 1800:1900:10]

In [ ]:
variable.data

After accessing the data, the variables attributes now contain the checksum

In [ ]:
variable.attributes

#### Masking values

In [ ]:
fill_value = int(variable.attributes['_FillValue'])
mask = (variable.data==fill_value)
numpy.ma.array(variable.data, mask=mask)

## Example 2: Groups and 64 bit floats
- Dataset containing groups
- Dataset containing 64 bit floats 

In [ ]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'test.opendap.org:8080'
path = '/opendap/dmrpp_test_files/'
dataset = 'ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'

### DAP2

In [ ]:
ds_dap2 = pydap.client.open_url(dap2_url)
ds_dap2

In [ ]:
ds_dap2._dict

In [ ]:
ds_dap2['/gt1r/bckgrd_atlas/delta_time']

In [ ]:
ds_dap2['/gt1r/bckgrd_atlas/delta_time'][0:10].data

### DAP4

In [ ]:
ds_dap4 = pydap.client.open_url(dap4_url)
ds_dap4

In [ ]:
ds_dap4._dict

In [ ]:
data = ds_dap4['/gt1r/bckgrd_atlas/delta_time'][0:20]
data

In [ ]:
data.attributes

## Example 3: Maps/Grids

In [ ]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'opendap.earthdata.nasa.gov'
path = '/hyrax/data/nc/'
dataset = 'coads_climatology.nc'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'

In [ ]:
ds_dap2 = pydap.client.open_url(dap2_url, session=session)
ds_dap4 = pydap.client.open_url(dap4_url, session=session)

In [ ]:
data_dap2 = ds_dap2['SST'][0:1:1, 40:42:1, 0:20:2]

In [ ]:
data_dap2.data

In [ ]:
data_dap4 = ds_dap4['SST'][0:1:1, 40:42:1, 0:20:2]

In [ ]:
data_dap4.data[:]

## Example 4 MOD05

In [ ]:
url = 'dap4://test.opendap.org/opendap/hyrax/data/stare/MOD05_L2.A2019336.2315.061.2019337071952.hdf'

In [ ]:
ds = pydap.client.open_url(url)

In [ ]:
ds['Water_Vapor_Infrared']

## Example 5 MY1DQND

In [ ]:
url = 'http://test.opendap.org/opendap/MODIS/MOOA/MY1DQN.004/2004.12.31/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf'

In [ ]:
ds = pydap.client.open_url(url)

In [ ]:
ds['sst_qual_b']